# Привет! Меня зовут Айрат, а это мой проект по анализу сервиса предоставления экскурсиий

#Первое задание
контекст:
В файле csv набор данных, в котором:
dimensions: event category, event action, event label
metrics: total events, unique events

●	1 строка = 1 вариант совершенного  события

●	event action - название действия события, где Page Visit - всегда посещение (= пользователь открыл страницу сайта), остальные события - всегда клики по элементам страницы (кнопки, чекбоксы)

в данном случае элементы - это фильтры на мобильной версии сайта, которые можно увидеть, например, на (чтобы посмотреть на компьютере, нужно перейти в мобильный режим)


	event_action	comment

0	Page Visit	посещение страницы города


1	search-tools-button_open	фильтры/сортировки/категории - открыть


2	filters-categories_click	сортировка и категории - активация чекбоксов


3	price_button_submit	применить фильтры


4	ticket-type_checkbox	фильтр мини/инди/группы


5	start_date_click	фильтр по датам(календарь) начало


6	end_date_click	фильтр по датам(календарь) конец


7	price_button_close	закрыть окошко с фильтрами


8	dates_filter_mobile	фильтр по датам(сегодня/завтра)


9	price_first	фильтр “стоимость"  - первый вариант


10	pay-type_checkbox	оплата на месте


11	clear_filter_mobile	очистить фильтры


12	price_second	фильтр “стоимость”  - второй вариант


13	price_third	фильтр “стоимость” - третий вариант


●	event label - дополнительная информация по событию
в лейбле на 1 месте всегда название города
там, где в лейбле через / написано что-то еще - это текст, по которому пользователь кликнул. там встречается много грязных данных
●	metrics: это количество всех (total events) и уникальных (unique events) событий, которые были затреканы на сайте

например:
City Page |	Page Visit |	Perm |	2353 |	1564
означает, что пользователей посетили страницу города Пермь 1564 уникальных раза (= кол-во сессий) и 2353 раза всего (за сессию 1 пользователь мог несколько раз быть на 1 странице)

например2:
price_button_submit |	Petrozavodsk / Показать предложения (38) |	6 |	6
означает, что пользователи на странице города Петрозаводск 6 раз нажали на кнопку “Показать предложения”, то есть применили фильтр. в момент этого нажатия именно у этих людей была надпись  Показать предложения (38), 38 в данном случае - кол-во предложений, выбранных фильтром

также на странице города Петрозаводск другие люди могли нажимать кнопку с событием price_button_submit, но event label мог быть другим
что сделать:
необходимо проанализировать использование фильтров пользователями на страницах городов. вопросы, на которые стоит ответить:
1.	сколько людей пользуются фильтрами?
2.	в каких городах фильтрами пользуются больше? в каких меньше?
3.	какие разделы фильтров наиболее востребованы? “фильтры”, “сортировка”, “категории”?
4.	как часто люди пользуются выбором цены?

не обязательно: любые дополнительные инсайты, полученные в результате исследования, будут плюсом к выполнению тестового задания.

*важно! в event label много грязных данных. оттуда можно убрать числа. там могут встречаться разные вещи. это нормально, поступай с этим так, как тебе покажется правильным


Работу условно разделил на 2 этапа:

*   1 этап: Анализ и преобразование исходных "грязных" данных
*   2 этап: Ответы на вопросы из задания

# 1 Этап

In [29]:
import pandas as pd
from google.colab import files
import re
from collections import namedtuple
from scipy.stats import binom, binomtest
import math
import numpy as np

In [30]:
uploaded = files.upload()

In [31]:
data_users = pd.read_csv('/content/events filters к тестовому Product Analyst Q1 2024.csv')

Смотрю, что по размерам таблицы у нас

In [32]:
data_users.shape

(19092, 5)

Как выглядят данные смотрим, чтобы понимать, что может тут быть не так

In [33]:
data_users.head(100)

,event_category,event_action,event_label,total_events,unique_events
0,city_landing,price_button_submit,Tula / Показать предложения (11),8,7
1,city_landing,price_button_submit,Sochi / Показать предложения (39),12,10
2,city_landing,search-tools-button_open,Penza / Сортировка,2,2
3,city_landing,filters-categories_click,Gelendzhik / ЭКСКУРСИИ В ГЕЛЕНДЖИКЕ ЦЕНЫ,1,1
4,city_landing,price_button_submit,Moscow / Показать предложения (345),2,2
...,...,...,...,...,...
95,city_landing,search-tools-button_open,Lazarevskoye / Сортировка По рейтингу,4,4
96,city_landing,price_button_submit,Antalya / Показать предложения (62),3,3
97,city_landing,price_button_submit,Moscow / Показать предложения (313),3,3
98,city_landing,price_button_submit,Bethlehem / Показать предложения (1),1,1


In [34]:
data_users.tail(10)

,event_category,event_action,event_label,total_events,unique_events
19082,city_landing,price_button_submit,Istanbul / Показать предложения (195),1,1
19083,city_landing,filters-categories_click,Moscow / ЭКСКУРСИИ МОСКВА,14,10
19084,city_landing,pay-type_checkbox,Alushta / Оплата на месте (без предоплаты),63,52
19085,city_landing,ticket-type_checkbox,Gorokhovets / Мини-группа,1,1
19086,city_landing,ticket-type_checkbox,Smolensk / Групповая,22,17
19087,city_landing,search-tools-button_open,Istanbul / ЭКСКУРСИИ В СТАМБУЛЕ,38,32
19088,city_landing,filters-categories_click,Ufa / Пешеходные,3,3
19089,city_landing,search-tools-button_open,Yerevan / Фильтры Выбрано: 0,16,16
19090,city_landing,price_button_submit,Pskov / Показать предложения (35),7,7
19091,City Page,Page Visit,Auckland,33,25


Быстрый поверхностный анализ на тип данных, пропуски, правильное ли количество уникальных значений (в колонке event_action), максимальные и минимальные значения

In [35]:
data_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19092 entries, 0 to 19091
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   event_category  19092 non-null  object
 1   event_action    19092 non-null  object
 2   event_label     19092 non-null  object
 3   total_events    19092 non-null  int64 
 4   unique_events   19092 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 745.9+ KB


In [36]:
data_users.describe(include='all')

,event_category,event_action,event_label,total_events,unique_events
count,19092,19092,19092,19092.000000,19092.000000
unique,2,14,16222,NaN,NaN
top,city_landing,price_button_submit,Venice,NaN,NaN
freq,18441,7354,8,NaN,NaN
mean,NaN,NaN,NaN,129.851456,87.966007
std,NaN,NaN,NaN,3392.922560,2216.777357
min,NaN,NaN,NaN,1.000000,1.000000
25%,NaN,NaN,NaN,1.000000,1.000000
50%,NaN,NaN,NaN,4.000000,4.000000
75%,NaN,NaN,NaN,14.000000,12.000000


Решил скопировать данные, чтобы в нём безопаснее работать

In [37]:
data_users_clean = data_users.copy()

Вынес название города, которое стоит до / в event_label, в отдельную колонку. Это нам пригодится по дальнейшим вопросам, да и часто в практике надо агрерировать по городам, поэтому такую колонку хочется иметь. Также оказалось, что некоторые города как магнитогорск с маленькой написаны, а в большинстве с заглавной буквы начинаются, поэтому тоже привёл к единому регистру.

In [38]:
data_users_clean['city'] = data_users_clean['event_label'].str.extract(r'^([^/]+)')
data_users_clean['city'] = data_users_clean['city'].str.strip()
data_users_clean['city'] = data_users_clean['city'].str.title()

Решил привести event_label к какому-то одному формату и убрать город, потому что это больше не нужно

In [39]:
def clean_label(label):
    label = re.sub(r'^.*?/', '', label)
    label = re.sub(r'\(\d+\)', '', label)
    label = re.sub(r'\d+', '', label)
    label = re.sub(r'[\/\.,:;!?—–\-]', '', label)
    label = label.strip()
    label = label.lower()
    return label

In [40]:
data_users_clean['event_label'] = data_users_clean['event_label'].apply(clean_label)

In [41]:
data_users_clean

,event_category,event_action,event_label,total_events,unique_events,city
0,city_landing,price_button_submit,показать предложения,8,7,Tula
1,city_landing,price_button_submit,показать предложения,12,10,Sochi
2,city_landing,search-tools-button_open,сортировка,2,2,Penza
3,city_landing,filters-categories_click,экскурсии в геленджике цены,1,1,Gelendzhik
4,city_landing,price_button_submit,показать предложения,2,2,Moscow
...,...,...,...,...,...,...
19087,city_landing,search-tools-button_open,экскурсии в стамбуле,38,32,Istanbul
19088,city_landing,filters-categories_click,пешеходные,3,3,Ufa
19089,city_landing,search-tools-button_open,фильтры выбрано,16,16,Yerevan
19090,city_landing,price_button_submit,показать предложения,7,7,Pskov


Посмотрел какие у нас значения есть для работы и как они выглядят

In [42]:
data_users_clean['event_category'].unique()

array(['city_landing', 'City Page'], dtype=object)

In [43]:
data_users_clean['event_action'].unique()

array(['price_button_submit', 'search-tools-button_open',
       'filters-categories_click', 'clear_filter_mobile',
       'start_date_click', 'ticket-type_checkbox', 'dates_filter_mobile',
       'price_third', 'Page Visit', 'end_date_click', 'price_second',
       'pay-type_checkbox', 'price_button_close', 'price_first'],
      dtype=object)

In [44]:
data_users_clean['event_label'].unique()

array(['показать предложения', 'сортировка',
       'экскурсии в геленджике цены', ..., 'turkistan',
       'категории экскурсии из тбилиси', 'отдых в адлере экскурсии'],
      dtype=object)

Проверил не потерял ли строки

In [45]:
data_users_clean.shape

(19092, 6)

Теперь у нас есть преобразованые и относительно чистые данные )

# 2 Этап

1.	Сколько людей пользуются фильтрами?

Уточнение я решил, что пользование фильтром буду считать только после нажатия кнопки применить фильтр. (уникальных пользователей)

Можно было по-другому, нажатие на любой фильтр считать за пользование фильтром, но мне эта идея не понравилась. Потому что один человек может нажимать на несколько фильтров и есть кнопка очистить фильтры.

In [46]:
answer_1 = data_users_clean[data_users_clean['event_action'] == 'price_button_submit'].unique_events.sum()

print(f"Количество людей, которые пользуются фильтрами: {answer_1} людей")

Количество людей, которые пользуются фильтрами: 63063 людей


2.  В каких городах фильтрами пользуются больше? в каких меньше?

Логика как в задаче выше, но группировали по городам

In [47]:
answer_2 = data_users_clean[data_users_clean['event_action'] == 'price_button_submit'].groupby('city').total_events.sum()
answer_2 = answer_2.reset_index().sort_values('total_events', ascending = True)

В этих городах мало смотрят экскурсии и прочее. В целом выглядит правдоподобно. Это либо непопулярные места или у спутника там мало предложений представлено.

In [48]:
answer_2[answer_2['total_events'] == 1]

,city,total_events
23,Baden-Baden,1
328,Valdai,1
321,Tuscany,1
336,Vicenza,1
315,Tolyatti,1
320,Tunis,1
43,Bibione,1
42,Bethlehem,1
69,Chemal,1
302,Sviyazhsk,1


В этих городах фильтрами пользуются больше всего. Выглядит тоже адекватно, в этих городах много трафика, предложений и фильтры там полезны.

In [49]:
answer_2.tail(10)

,city,total_events
282,Sharm-El-Sheikh,1661
13,Anapa,1837
141,Kazan,2072
55,Budapest,2154
83,Dubai,2406
126,Istanbul,2778
290,Sochi,3286
199,Moscow,3598
265,Saint Petersburg,5601
134,Kaliningrad,6113


3.	Какие разделы фильтров наиболее востребованы? “фильтры”, “сортировка”, “категории”?

Вот тут расчёт немного изменил я. Теперь считал не уникальные нажатия, а все нажатия (вопрос стоит в восстребованности и думаю такое справедливо). Стоял вопрос, что относить к фильтрам, что к категориям, что к сортировкам.

Логика примерно следующая:

* Фильтры скрывают всё, что не подходит.

('start_date_click','end_date_click','dates_filter_mobile','price_first','price_second','price_third')

* Категории помогают выбрать “тип” того, что интересует.

('ticket-type_checkbox','pay-type_checkbox','filters-categories_click')

* Сортировка показывает "что сначала", но не меняет состав результатов.

('filters-categories_click')

In [50]:
answer_filter = data_users_clean[data_users_clean['event_action'].isin(['start_date_click','end_date_click','dates_filter_mobile','price_first','price_second','price_third'])]

print(f"Сколько раз нажимали на фильтры: {answer_filter.total_events.sum()}")

Сколько раз нажимали на фильтры: 59917


In [51]:
answer_category = data_users_clean[data_users_clean['event_action'].isin(['ticket-type_checkbox','pay-type_checkbox','filters-categories_click'])]

print(f"Сколько раз нажимали на категории: {answer_category.total_events.sum()}")

Сколько раз нажимали на категории: 110268


In [52]:
answer_sort = data_users_clean[data_users_clean['event_action'].isin(['filters-categories_click'])]

print(f"Сколько раз нажимали на сортировки: {answer_sort.total_events.sum()}")

Сколько раз нажимали на сортировки: 80065


4.	Как часто люди пользуются выбором цены?

In [53]:
count_users = data_users_clean[data_users_clean['event_action'].isin(['Page Visit'])].unique_events.sum()

answer_price = data_users_clean[data_users_clean['event_action'].isin(['price_first','price_second','price_third'])]

price_users = answer_price.unique_events.sum()
price_users
print(f"Доля пользователей, которые пользуются выбором цен: {round(price_users/ count_users, 3) * 100} %")

Доля пользователей, которые пользуются выбором цен: 0.7000000000000001 %


#  Второе задание
Предположим, что сервис хочет добавить на сайт новый фильтр по времени начала экскурсий. Мы хотим провести А/Б-тест, чтобы узнать, увеличит ли этот новый фильтр количество бронирований

Вопросы по тесту:

1. Опиши, какой результат мы хотим достичь с помощью нового фильтра, на какие метрики стоит смотреть.

Допустим, после проведения теста на протяжении недели, мы получили следующие результаты:
●	В группе А (без нового фильтра) из 5000 пользователей бронирование совершили 300 человек.
●	В группе Б (с новым фильтром) из 5000 пользователей бронирование совершили 450 человек.

2. Подсчитай конверсию в бронирование для каждой группы.
3. Оцени, насколько новый фильтр повлиял на увеличение бронирований.
4. Стоит ли внедрять данный фильтр на основании полученных результатов?


1. Опиши, какой результат мы хотим достичь с помощью нового фильтра, на какие метрики стоит смотреть.

Цель: Увеличение количества бронирований

Целевая метрика: Конверсия =  число бронирований / число пользователей

Прокси-метрики:
1. Доля пользователей, применивших фильтр
2. Длительность сессии (лучше туры смотрят)
3. Количество просмотров туров

Контр-метрики:
1. CSAT
2. Можно время смотреть (вдруг слишком долго смотрят, значит мы усложнили выбор)
3. Тех. сбои


2. Подсчитай конверсию в бронирование для каждой группы.

In [54]:
group_a_conversion = 300 / 5000
group_b_conversion = 450 / 5000
print(f"Конверсия группы А: {group_a_conversion}")
print(f"Конверсия группы B: {group_b_conversion}")

Конверсия группы А: 0.06
Конверсия группы B: 0.09


3. Оцени, насколько новый фильтр повлиял на увеличение бронирований.

In [55]:
print(f"Абсолютный рост: {(group_b_conversion - group_a_conversion) * 100}")
print(f"Относительный рост: {(group_b_conversion - group_a_conversion) / group_a_conversion * 100}")

Абсолютный рост: 3.0
Относительный рост: 50.0


Интересно, относительный рост 50 % это много...

Вот мои мысли. Я бы попробовал проверить это биноминальным тестом. Можем использовать бином, поскольку у нас конверсия и есть исход пользовател сконвертировался (1) и не сконвертировался (0).

 - $H_0$: Конверсия в бронирование 0.06
 - $H_1$: Конверсия в бронирование > 0.06

Успешность каждого отдельного бронирования - это случайная величина $\xi$ из распределения Бернулли. Где забронировал 1, не забронировал 0.
$$ \xi = \xi_1, \xi_2, \dots, \xi_{5000} \sim Bern(\mu) - \text{выборка} $$

$$ \mu - \text{неизвестный параметр} $$
Статистика критерия считается, как:

$$Q = \sum \xi_i$$
$$Q\sim  Binom(0.06, 5000)$$

 $C_{rightcrit}$ -  критическое значение правого хвоста распределения и тогда критерий будет:

$$ S = \{Q(\xi) \geqslant C_{rightcrit}\}$$


P_value для критерия.
$p \leqslant \alpha$. $\alpha$ это у нас уровень значимости, возьмем базово 0.05. Тогда критерий отклонения по $p_value$ будет следующий:

$$ P(C_{rightcrit} \leqslant \xi) \leqslant \alpha$$


In [56]:
BinCheckResults = namedtuple('BinCheckResults', ['is_rejected', 'pvalue'])

def check_binom(n: int, k: int, alpha: float = 0.05):
    """
    Параметры:
    - n: количество людей
    - k: количество бронирований
    - alpha: уровень значимости критерия.

    Возвращает:
        - отверглась или нет гипотеза H_0 на уровне значимости alpha
        - pvalue: float
    """

    is_rejected = None
    pvalue = None

    binom_H0 = binom(n, p = group_a_conversion)
    pvalue = 1 - binom_H0.cdf(k - 1)
    is_rejected = pvalue <= alpha

    return BinCheckResults(is_rejected, pvalue)

In [57]:
check_binom(n = 5000, k = 450)

BinCheckResults(is_rejected=np.True_, pvalue=np.float64(0.0))

In [58]:
binomtest(450, 5000, p=0.06, alternative='greater')

BinomTestResult(k=450, n=5000, alternative='greater', statistic=0.09, pvalue=3.910193915335487e-17)

4. Стоит ли внедрять данный фильтр на основании полученных результатов?

Проверил встроенным биномтестом и сам функцию расписал, итог один. H0 отвергаем, в пользу H1. Стоит внедрять фильтр, так-то и на основнии относительного прироста думаю могли, 50% это очень много, если данные правильно записывались.